In [9]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
import gc
import time
import sys
import datetime
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
from sklearn import metrics

from catboost import Pool, CatBoostClassifier



In [10]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

model = CatBoostClassifier(loss_function="Logloss", eval_metric="AUC")
kf = KFold(n_splits=5, random_state=42, shuffle=True)

y_valid_pred = 0 * target
y_test_pred = 0

for idx, (train_index, valid_index) in enumerate(kf.split(train_df)):
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    X_train, X_valid = train_df[features].iloc[train_index,:], train_df[features].iloc[valid_index,:]
    _train = Pool(X_train, label=y_train)
    _valid = Pool(X_valid, label=y_valid)
    print( "\nFold ", idx)
    fit_model = model.fit(_train,
                          eval_set=_valid,
                          use_best_model=True,
                          verbose=200
                         )
    pred = fit_model.predict_proba(X_valid)[:,1]
    print( "  auc = ", roc_auc_score(y_valid, pred) )
    y_valid_pred.iloc[valid_index] = pred
    y_test_pred += fit_model.predict_proba(test_df[features])[:,1]
y_test_pred /= 5

sub_df1 = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df1["target"] = y_test_pred
sub_df1.to_csv("submission.csv", index=False)



Fold  0
Learning rate set to 0.132949
0:	test: 0.5315318	best: 0.5315318 (0)	total: 139ms	remaining: 2m 18s
200:	test: 0.8831111	best: 0.8831111 (200)	total: 21.4s	remaining: 1m 25s
400:	test: 0.8924405	best: 0.8924405 (400)	total: 43.2s	remaining: 1m 4s
600:	test: 0.8951229	best: 0.8951415 (598)	total: 1m 4s	remaining: 42.9s
800:	test: 0.8953375	best: 0.8954127 (786)	total: 1m 26s	remaining: 21.4s
999:	test: 0.8955113	best: 0.8955990 (924)	total: 1m 47s	remaining: 0us

bestTest = 0.8955990231
bestIteration = 924

Shrink model to first 925 iterations.
  auc =  0.8955990230803251

Fold  1
Learning rate set to 0.132949
0:	test: 0.5274413	best: 0.5274413 (0)	total: 119ms	remaining: 1m 58s
200:	test: 0.8836085	best: 0.8836085 (200)	total: 21.9s	remaining: 1m 27s
400:	test: 0.8941822	best: 0.8942216 (399)	total: 44.4s	remaining: 1m 6s
600:	test: 0.8971854	best: 0.8971897 (599)	total: 1m 6s	remaining: 44s
800:	test: 0.8975168	best: 0.8977881 (710)	total: 1m 27s	remaining: 21.8s
999:	test: 0

KeyboardInterrupt: 